In [15]:
import google.generativeai as genai
import os
import json
import time
from typing import List, Dict, Any, Optional

import google.generativeai as genai
from PIL import Image
import os
from dotenv import load_dotenv

load_dotenv()

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

if GOOGLE_API_KEY is None:
    raise ValueError("GOOGLE_API_KEY not found in environment variables. Please set it in your .env file.")

genai.configure(api_key=GOOGLE_API_KEY)


class SearchTimeoutError(Exception):
    pass

class MilvusConnectionError(Exception):
    pass

def search_products(
    text: str,
    image: bool = False,
    image_url: Optional[List[str]] = None,
    filters: Optional[Dict[str, Any]] = None,
    attempt_number: int = 1,
    original_query: str = ""
) -> Dict[str, Any]:
    """
    Search for products in the Milvus vector database using hybrid search.

    This tool performs semantic search across product titles, descriptions, and tags
    using both text and image embeddings when applicable.

    Args:
        text: Search query text combining user intent and extracted terms
        image: Whether image search is included in this request
        image_url: List of image URLs to process for visual search
        filters: Dictionary containing search filters:
            - category: Product category (e.g., "Furniture", "Lamps")
            - price_range: {"min": float, "max": float, "operation": "range|loe|hoe|eq"}
            - attributes: {"color": str, "size_": str, etc.}
        attempt_number: Current retry attempt (1-5)
        original_query: Original user query for context

    Returns:
        Dictionary with:
        - success: bool
        - products: List of up to 5 matching products
        - total_found: int
        - search_time: float
        - attempt_info: Dict with attempt details
        - error: str (if any)

    Raises:
        SearchTimeoutError: If search exceeds 120 seconds
        MilvusConnectionError: If database is unavailable
    """

    start_time = time.time()

    try:
        print(f"[SEARCH_PRODUCTS] Attempt {attempt_number}: Searching for '{text}'")
        print(f"[SEARCH_PRODUCTS] Filters: {json.dumps(filters, indent=2)}")
        print(f"[SEARCH_PRODUCTS] Image search: {image}, URLs: {image_url}")

        # Simulate search timeout check
        if start_time > time.time() + 120:  # This is just for demo - replace with actual timeout logic
            raise SearchTimeoutError("Search operation timed out after 120 seconds")

        # TODO: Replace this with your actual Milvus search implementation
        # This is a mock implementation for testing
        mock_products = [
            {
                "id": f"prod_{attempt_number}_1",
                "name": f"Modern {text.split()[0] if text.split() else 'Product'} - Style A",
                "price": 299.99,
                "category": filters.get("category", "General") if filters else "General",
                "score": 0.95,
                "attributes": {"color": "white", "material": "wood"},
                "product_odoo_id": 12345,
                "variant_odoo_id": 67890
            },
            {
                "id": f"prod_{attempt_number}_2",
                "name": f"Premium {text.split()[0] if text.split() else 'Product'} - Style B",
                "price": 449.99,
                "category": filters.get("category", "General") if filters else "General",
                "score": 0.87,
                "attributes": {"color": "black", "material": "metal"},
                "product_odoo_id": 12346,
                "variant_odoo_id": 67891
            }
        ]

        # Simulate different results based on attempt number
        if attempt_number > 1:
            # Simulate adaptive search results
            mock_products[0]["name"] = f"Alternative {text.split()[0] if text.split() else 'Product'} - Attempt {attempt_number}"
            mock_products[0]["score"] = max(0.5, 0.95 - (attempt_number * 0.1))

        execution_time = time.time() - start_time

        result = {
            "success": True,
            "products": mock_products,
            "total_found": len(mock_products),
            "search_time": execution_time,
            "attempt_info": {
                "attempt_number": attempt_number,
                "original_query": original_query,
                "adapted_text": text,
                "filters_used": filters
            }
        }

        print(f"[SEARCH_PRODUCTS] Found {len(mock_products)} products in {execution_time:.2f}s")
        return result

    except SearchTimeoutError as e:
        return {
            "success": False,
            "error": str(e),
            "products": [],
            "total_found": 0,
            "search_time": time.time() - start_time,
            "attempt_info": {"attempt_number": attempt_number, "timeout": True}
        }
    except MilvusConnectionError as e:
        return {
            "success": False,
            "error": "Product database is temporarily unavailable",
            "products": [],
            "total_found": 0,
            "search_time": time.time() - start_time,
            "attempt_info": {"attempt_number": attempt_number, "milvus_down": True}
        }
    except Exception as e:
        return {
            "success": False,
            "error": f"Unexpected error: {str(e)}",
            "products": [],
            "total_found": 0,
            "search_time": time.time() - start_time,
            "attempt_info": {"attempt_number": attempt_number, "general_error": True}
        }



genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

model = genai.GenerativeModel('gemini-2.5-flash', tools=[search_products])

chat = model.start_chat(enable_automatic_function_calling=False)

response = chat.send_message("I'm looking for a red sofa for my living room, preferably under $500.")

print("\n--- Model Response ---")
print(response)


--- Model Response ---
response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "function_call": {
                  "name": "search_products",
                  "args": {
                    "text": "red sofa",
                    "filters": {
                      "attributes": {
                        "color": "red"
                      },
                      "price_range": {
                        "operation": "loe",
                        "max": 500.0
                      }
                    }
                  }
                }
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "index": 0
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 424,
        "candidates_token_count": 47,
        "total_token_count": 647
   

In [11]:
try:
    movie_function_declaration = types.FunctionDeclaration.from_callable(
        callable = find_movies,
        client = client 
    )

    declaration_dict = movie_function_declaration.to_json_dict()

    # 4. Print the schema using the json library for pretty formatting
    print("--- Function Declaration Schema for 'find_movies' ---")
    print(json.dumps(declaration_dict, indent=4))

except Exception as e:

    print(f"Note: Could not generate schema dynamically ({e}). Displaying expected schema:")
    mock_schema = {
        "name": "find_movies",
        "description": "Find movie titles currently playing in theaters based on any description, genre, title words, etc.\n\nArgs:\n    description: Any kind of description including category or genre, title words, attributes, etc.\n    location: The city and state, e.g. San Francisco, CA or a zip code e.g. 95616",
        "parameters": {
            "type": "OBJECT",
            "properties": {
                "description": {
                    "type": "STRING"
                },
                "location": {
                    "type": "STRING"
                }
            },
            "required": [
                "description",
                "location"
            ]
        }
    }
    print(json.dumps(mock_schema, indent=4))


Note: Could not generate schema dynamically (type object 'FunctionDeclaration' has no attribute 'from_callable'). Displaying expected schema:
{
    "name": "find_movies",
    "description": "Find movie titles currently playing in theaters based on any description, genre, title words, etc.\n\nArgs:\n    description: Any kind of description including category or genre, title words, attributes, etc.\n    location: The city and state, e.g. San Francisco, CA or a zip code e.g. 95616",
    "parameters": {
        "type": "OBJECT",
        "properties": {
            "description": {
                "type": "STRING"
            },
            "location": {
                "type": "STRING"
            }
        },
        "required": [
            "description",
            "location"
        ]
    }
}
